In [35]:
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from xgboost import XGBClassifier
#pd.set_option('display.max_columns', 200)
#pd.set_option('display.max_colwidth', 300)

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
inputs = [
        'bad product',
        'Bad product',
        'I read through the reviews on here before looking in to buying one of the couples lubricants, and was ultimately disappointed that it didnt even live up to the reviews I had read. For starters, neither my boyfriend nor I could notice any sort of enhanced or captivating sensation. What we did notice, however, was the messy consistency that was reminiscent of a more liquid-y vaseline. It was difficult to clean up, and was not a pleasant, especially since it lacked the captivating sensation we had both been expecting. I am disappointed that I paid as much as I did for a lube that I wont use again, when I could just use their normal personal lubricant for 1) less money and 2) less mess.',
       'Loved it, I am using the oneplus 8T I got this phone for my girlfriend and even though I spent like 13k extra when I got 8T I dont feel like the difference between both the phones are worth 13k. I am loving this phone.',
       'Sleepy service, poor food quality, and when we asked why it was like this they stated that their kitchen was backed up, yet the restaurant was damn near empty.',
       'Took an hour to get food. Food was dry and not cooked well and my friend found hair in her food. It was a busy night I get it, but worst service I have ever experienced. Go ANYWHERE else.',
       'Bad food quality, wastage of money, we can get better food in the other hotels.',
       'The product is very bad',
       'I never seen such a worst product from last 2 years',
       'Each slice was inconsistent with toppings and how cooked it was. Salad was so rotted I had to throw it away or risk food poisoning. The pasta looked crusty so I stayed away should have just stayed away from this place altogether. Awful. Anyone with more than a one star review was paid for sure.',
       'Awful. Service from start to finish was not good. Food was okay. Paid for a meal for 3, & immediately after finishing our food we were continuously asked to leave our table. When we said no (because we paid to be there) we were met with a guilt trip, rolled eyes, and heavy sighs.']
#inputs = ['The product is good', 'The product is good']
simple_test = pd.DataFrame(inputs,columns =['reviews'])
#simple_test.reviews


In [38]:
loaded_model = pickle.load(open("drive/MyDrive/model.pickle", "rb"))
loaded_vectorizer = pickle.load(open('drive/MyDrive/vectorizer.pickle', 'rb'))
#sentiment_predict(simple_test)

In [39]:
def sentiment_predict(reviews_text):
  test_data = TfidfVectorizer(vocabulary=loaded_vectorizer.get_feature_names_out())
  X_test_value = test_data.fit_transform(reviews_text)
  sentiment_output = pd.DataFrame(loaded_model.predict(X_test_value), columns=['sentiment'])
  reviews_text.reset_index(drop=True, inplace=True)
  sentiment_output.reset_index(drop=True, inplace=True) 
  sentiment_output = pd.concat([reviews_text, sentiment_output], axis=1)
  return sentiment_output

In [40]:
# load model from file
itembaseddf = pickle.load(open("drive/MyDrive/itembased-df.pickle", "rb"))

In [41]:
itembaseddf

name,0.6 Cu. Ft. Letter A4 Size Waterproof 30 Min. Fire File Chest,100:Complete First Season (blu-Ray),2017-2018 Brownline174 Duraflex 14-Month Planner 8 1/2 X 11 Black,"2x Ultra Era with Oxi Booster, 50fl oz","42 Dual Drop Leaf Table with 2 Madrid Chairs""",4C Grated Parmesan Cheese 100% Natural 8oz Shaker,Africa's Best No-Lye Dual Conditioning Relaxer System Super,Alberto VO5 Salon Series Smooth Plus Sleek Shampoo,"All,bran Complete Wheat Flakes, 18 Oz.",Ambi Complexion Cleansing Bar,Annie's Homegrown Deluxe Elbows & Four Cheese Sauce,Annie's Homegrown Gluten Free Double Chocolate Chip Granola Bars,Arrid Extra Dry Anti-Perspirant Deodorant Spray Regular,"Aussie Aussome Volume Shampoo, 13.5 Oz","Australian Gold Exotic Blend Lotion, SPF 4","Aveeno Baby Continuous Protection Lotion Sunscreen with Broad Spectrum SPF 55, 4oz","Avery174 Ready Index Contemporary Table Of Contents Divider, 1-8, Multi, Letter",Axe Dry Anti-Perspirant Deodorant Invisible Solid Phoenix,"Banana Boat Sunless Summer Color Self Tanning Lotion, Light To Medium",Barielle Nail Rebuilding Protein,Baxter Of California Cream Pomade,Bedtime Originals Pinkie Musical Mobile,"Ben & Jerry's Coffee, Coffee Buzzbuzzbuzz! Ice Cream, Pint",Bi-O-kleen Spray & Wipe All Purpose Cleaner,Bilbao Nightstand Gray Oak - South Shore,Bill Glor Gaither - Church In The Wildwood (cd),Bisquick Original Pancake And Baking Mix - 40oz,Black Front Loading Frame Set (8.5x11) Set Of 12,Black Sister's Revenge (dvd),Blue Anchor Design Throw Pillow (18x18) - Rizzy Home,Boraam Sonoma Kitchen Cart With Wire Brush Gray - Maaya Home,"Bounce Dryer Sheets, Fresh Linen, 160 sheets",Bumble Bee Solid White Albacore In Water - 5 Oz,"Burt's Bees Lip Shimmer, Raisin","Burt's Bees Lip Shimmer, Watermelon",Cal Lighting Led Dark Bronze Finish Metal Piano Lamp,Calvin Klein Secret Obsession Eau De Parfum,"Candy Pink Plastic Cups, 20 pk",Cantu Coconut Milk Shine Hold Mist - 8oz,Care Free Curl Gold Instant Activator,...,Starbucks Iced Expresso Classics Vanilla Latte Coffee Beverage - 40oz,Starbucks153 Doubleshot Protein Coffee - 11 Fl Oz,Stargate (ws) (ultimate Edition) (director's Cut) (dvdvideo),"Storkcraft Tuscany Glider and Ottoman, Beige Cushions, Espresso Finish","Suave Professionals Hair Conditioner, Sleek",Sunflower Swag With Metal Frame - Nearly Natural,"Super Poligrip Denture Adhesive Cream, Ultra Fresh - 2.4 Oz",Switchmas (dvd),Tai Pei Sweet & Sour Chicken,The Honest Company Laundry Detergent,The Resident Evil Collection 5 Discs (blu-Ray),The Script - No Sound Without Silence (cd),"The Seaweed Bath Co. Argan Conditioner, Smoothing Citrus",There's Something About Mary (dvd),Tim Holtz Retractable Craft Pick-Red 6x.5,Tostitos Bite Size Tortilla Chips,Tostitos Original Restaurant Style Tortilla Chips,Tostitos Simply Blue Corn Tortilla Chips,Toy Story Kids' Woody Accessory Kit,Tramontina Ceramica 10 Piece Cookware Set - Red,"Tree Hut Shea Body Butters, Coconut Lime, 7 oz",Trend Lab Park Nursing Cover - Paisley,Tresemme Kertatin Smooth Infusing Conditioning,Udi's Pepperoni Pizza,Various - Country's Greatest Gospel:Gold Ed (cd),Various - Red Hot Blue:Tribute To Cole Porter (cd),Various Artists - Choo Choo Soul (cd),Vaseline Intensive Care Healthy Hands Stronger Nails,Vaseline Intensive Care Lip Therapy Cocoa Butter,"Vicks Vaporub, Regular, 3.53oz",Voortman Sugar Free Fudge Chocolate Chip Cookies,Wagan Smartac 80watt Inverter With Usb,Walkers Stem Ginger Shortbread,"Wallmount Server Cabinet (450mm, 9 RU)","Way Basics 3-Shelf Eco Narrow Bookcase Storage Shelf, Espresso - Formaldehyde Free - Lifetime Guarantee",Wedding Wishes Wedding Guest Book,Weleda Everon Lip Balm,Windex Original Glass Cleaner Refill 67.6oz (2 Liter),Yes To Carrots Nourishing Body Wash,Yes To Grapefruit Rejuvenating Body Wash
reviews_username,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,

In [42]:
ratings = pd.read_csv('drive/MyDrive/clean_text.csv' , encoding='latin-1')
ratings.head()

,Unnamed: 0,id,brand,categories,manufacturer,name,reviews_date,reviews_rating,reviews_text,reviews_title,reviews_username,user_sentiment
0,0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,5,love album good hip hop side current pop sound...,Just Awesome,joshua,1
1,1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,5,good flavor this review collected part promotion,Good,dorothy w,1
2,2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,5,good flavor,Good,dorothy w,1
3,3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,1,i read reviews looking buying one couples lubr...,Disappointed,rebecca,0
4,4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,1,my husband bought gel us the gel caused irrita...,Irritation,walker557,0


In [43]:
#Recommending top 20 products
top_recommended_products = pd.DataFrame(itembaseddf.loc['08dallas'].sort_values(ascending=False)[0:20])
sentimetment_result_df = pd.DataFrame()
for index, row in top_recommended_products.iterrows():
  product_reviews = ratings[ratings['name'].isin([row.name])].reviews_text
  #print(len(product_reviews))
  sentimetment_result = sentiment_predict(product_reviews)
  sentimetment_result['name'] = row.name
  sentimetment_result_df = sentimetment_result_df.append(sentimetment_result)
  #print(sentimetment_result_df.shape)

In [44]:
gb = sentimetment_result_df.groupby(['name','sentiment']).size().reset_index(name='counts')

In [45]:
gb_total = sentimetment_result_df.groupby(['name']).size().reset_index(name='total_counts')

In [46]:
gb = pd.merge(gb,gb_total[['name','total_counts']],on='name', how='right')

In [47]:
gb['percentage'] = (gb['counts']/gb['total_counts'])*100

In [48]:
gb_final = gb[(gb['percentage'] >=50) & (gb['sentiment'] == 1)]

In [49]:
gb_final.sort_values('percentage', ascending=False).head(5)

,name,sentiment,counts,total_counts,percentage
21,"Heinz Tomato Ketchup, 38oz",1,1,1,100.000000
23,Herr's Baked Cheese Curls,1,1,1,100.000000
14,Happy Tot Apple and Butternut Squash,1,9,9,100.000000
22,Heiress (dvd),1,2,2,100.000000
1,Clorox Disinfecting Bathroom Cleaner,1,1992,2039,97.694949


In [50]:
pip freeze

absl-py==0.12.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argcomplete==2.0.0
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.11.4
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==4.1.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.2
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.10
cached-property==1.5.2
cachetools==4.2.4
catalogue==1.0.0
certifi==2021.10.8
cffi==1.15.0
cftime==1.5.1.1
chardet==3.0.4
charset-normalizer==2.0.10
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.3.2
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.26
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dil